**Import Resources**

In [1]:
# importing resources
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#import ipywidgets as widgets
#from ipywidgets import interact, interact_manual

**Select function from User input**

In [2]:
# Setting function from User input
func = input('Choose the function name \n(Options: Linear, Cubic, Sine, TanH, XOR, Complex) = ')

Choose the function name 
(Options: Linear, Cubic, Sine, TanH, XOR, Complex) = Complex


In [ ]:
## try and except statment
#handling errors & exceptions

**Set Hyperparameters from User input**

In [3]:
# Setting Hyperparameters from User input

# set number of hidden layers
hidden_layers = int(input('Enter number of hidden layers = '))

# set number of nodes/neurons in the hidden layers
if(hidden_layers > 0):
    hidden_layer_nodes = []
    for layer in range(hidden_layers):
        hidden_layer_nodes.append(int(input('Enter number of nodes in hidden layer {}= '.format(layer+1))))
    
# set activation function
activation = input('Enter the activation function \n(Options: Null, Sigmoid, Hyperbolic Tangent, Cosine, Gaussian) = ')
    
#set learning rate
learning_rate = float(input('Enter value of learning rate = '))

#check
print(hidden_layer_nodes)
print(activation)
print(learning_rate)


Enter number of hidden layers = 5
Enter number of nodes in hidden layer 1= 10
Enter number of nodes in hidden layer 2= 8
Enter number of nodes in hidden layer 3= 6
Enter number of nodes in hidden layer 4= 4
Enter number of nodes in hidden layer 5= 3
Enter the activation function 
(Options: Null, Sigmoid, Hyperbolic Tangent, Cosine, Gaussian) = Gaussian
Enter value of learning rate = 0.1
[10, 8, 6, 4, 3]
Gaussian
0.1


###### **Set number of nodes in input and output layers based on the selected function**

In [ ]:
'''
# Setting number nodes/neurons in the Input layer
input_nodes = 1 #input nodes=1 for Linear, Cubic, Sine, Tanh since these functions are based on single input variable

if func in ('XOR','Complex'):
    input_nodes = 2 # Since XOR & Complex functions are based on two input variables
   
    
# Setting number of nodes/neurons in the Output layer
output_nodes =1 # Since only single output value is expected from the fuction

#check
print(input_nodes, output_nodes)
'''

In [4]:
activation_funcs = {
    'Null': lambda x: 0,
    'Sigmoid': lambda x: 1/(1 + np.exp(-x)),
    'Hyperbolic Tangent': lambda x: np.tanh(x),
    'Cosine': lambda x: np.cos(x),
    'Gaussian': lambda x: np.exp(-x**2/2),
        }

In [ ]:
#check only
activate = activation_funcs[activation]
print ('Output: Selected activation -',activate(.5))

print('Output: Null -',0)
print('Output: Sigmoid -',1/(1 + np.exp(-.5)))
print('Output: Hyperbolic Tangent -',np.tanh(.5))
print('Output: Cosine -',np.cos(.5))
print('Output: Gaussian -',np.exp(-.5**2/2))

In [ ]:
#check only
for i in range(len(hidden_layer_nodes)):
    print("number of neurons in hidden layer {} is {}".format(i+1,hidden_layer_nodes[i]))

**Create Neural Network Class**

In [5]:
class neuralNetwork:    
    def __init__(self, func, hiddenLayerNodes, learningRate, activation):
        self.func = func
        self.layerArch = hiddenLayerNodes
        self.lr = learningRate
        self.activation = activation
        
        # setting layer architecture as a list
        self.layerArchitecture()
        
        #intialize the weight array
        self.weights = self.allWeights()

        self.actfunc = self.functionSelection()
        
    def layerArchitecture(self):
        # Setting number nodes/neurons in the Input layer
        if self.func in ('XOR','Complex'):
            self.layerArch.insert(0,2) # XOR & Complex functions are based on two input variables
        else:
            self.layerArch.insert(0,1) # Linear, Cubic, Sine, Tanh functions are based on single input variable
       
        # Setting number of nodes/neurons in the Output layer
        self.layerArch.append(1) # only single output value is expected from the fuction
        
    
    def allWeights(self):
        n_weights = sum(self.layerArch[i]*self.layerArch[i+1] for i in range(len(self.layerArch)-1))
        return np.random.rand(n_weights)
    
    def functionSelection(self):
        return activation_funcs[self.activation]

    def forward(self, input_list):
        splitIndices = []
        for i in range(len(self.layerArch)-1):
            splitIndices.append(self.layerArch[i]*self.layerArch[i+1])
            if i != 0:
                splitIndices[i] = splitIndices[i]+splitIndices[i-1]
               
        wArray = np.split(self.weights,splitIndices)
        
        input_array = np.asarray(input_list)
        for i in range(len(self.layerArch)-1):
            a = wArray[i].reshape(self.layerArch[i+1],self.layerArch[i])
            
            input_array = np.matmul(a,input_array)
            if i != (len(self.layerArch)-1):
                input_array = self.actfunc(input_array)
        
        final_output = input_array.tolist()
            
        #inputs = np.array(inputs_list,ndmin = 1).T
                
        return final_output

In [6]:
#create instance of neural network
nn = neuralNetwork(func, hidden_layer_nodes, learning_rate, activation)


**Reading data from the .txt file**

In [7]:
# load the training data from .txt file into a list
txt_address = {'Linear':'cw_dataset/1in_linear.txt',
               'Cubic':'cw_dataset/1in_cubic.txt',
               'Sine':'cw_dataset/1in_sine.txt',
               'TanH':'cw_dataset/1in_tanh.txt',
               'XOR':'cw_dataset/2in_xor.txt',
               'Complex':'cw_dataset/2in_complex.txt'}

train_data_file = open(txt_address[func],'r')
line_data = train_data_file.readlines()
train_data_file.close()

train_list = []
for line in line_data:
    line = line.strip().split()
    for d in line:
        train_list.append(float(d))
        
#check     
#print(train_list)

#train_array = np.array(train_list, dtype=np.float32).reshape((2,200))
#train_array = np.array(train_list).reshape((-1,2))
#X,Y = train_array[:,0],train_array[:,-1]

if func in ('XOR','Complex'):
    X1,X2,Y = train_list[::3],train_list[1::3],train_list[2::3]
else:
    X,Y = train_list[::2],train_list[1::2]

In [ ]:
yOut = []
for i in range(len(X)):
    yOut.append(nn.forward([X[i]]))

In [8]:
yOut = []
for i in range(len(X1)):
    yOut.append(nn.forward([X1[i],X2[i]]))

In [9]:
print(yOut)

[[0.24989900014167088], [0.24996226328246407], [0.24989900066801302], [0.24989900357188283], [0.24989914171796965], [0.24989917915724127], [0.2499013381092943], [0.24989900310163574], [0.24989900073688895], [0.24989905951150246], [0.24989900115476565], [0.24989900013354885], [0.24989900145494756], [0.24989974748742066], [0.24990475093661915], [0.24989988318588166], [0.2498990001638481], [0.24989900014989633], [0.24989900013230595], [0.24989902472343958], [0.2498990001295278], [0.24989901386632102], [0.24989915825723513], [0.2498990001457833], [0.24989900027684436], [0.2498990001386761], [0.24989900033632628], [0.24989936335727142], [0.24993659037429747], [0.24989900041270652], [0.2498990007973466], [0.2498990001205781], [0.24989911833882295], [0.2499376278488227], [0.24989900048952854], [0.2498990002054136], [0.24990003508969047], [0.24989954742041576], [0.24989958024538841], [0.24989927565610298], [0.24990532601198376], [0.24989900026164366], [0.24989903346081294], [0.2498992163736019

In [10]:
print(X1)
print(len(X1))
print(X2)
print(len(X2))

[0.6771, -0.9142, 0.1488, 0.6137, -0.4975, 0.3031, 0.6318, -0.3801, 0.0729, -0.578, 0.3036, -0.5414, -0.3781, 0.4414, -0.7377, -0.7495, 0.8228, 0.234, -0.5587, 0.098, -0.3403, 0.8869, 0.7603, -0.2408, -0.6744, -0.6075, 0.2266, -0.9379, 0.9421, -0.5439, 0.3597, 0.1997, -0.5693, -0.8465, 0.5776, 0.7559, 0.4443, -0.6887, -0.3732, -0.2847, 0.5873, -0.7795, -0.7349, -0.7304, -0.2069, -0.5179, -0.2178, -0.8142, -0.6809, 0.7583, 0.9826, 0.7427, -0.008, -0.8781, -0.6275, -0.7534, -0.2606, 0.7787, -0.6866, -0.5332, -0.2062, -0.83, 0.9394, 0.5639, -0.6085, 0.4276, 0.2742, 0.6961, 0.3366, 0.6413, -0.0262, 0.2831, 0.3219, 0.8764, -0.818, 0.0298, 0.45, 0.9282, -0.6778, -0.9995, -0.4498, -0.8122, 0.6338, -0.9549, 0.696, 0.9102, 0.4846, 0.768, -0.881, 0.9164, -0.1673, -0.1001, -0.2168, -0.2912, 0.3017, 0.6656, 0.2597, 0.0844, 0.7262, -0.0555]
100
[-0.4831, -0.9882, 0.4878, 0.2751, -0.7114, 0.8922, 0.8605, -0.4624, -0.6734, -0.5664, -0.8944, 0.3349, -0.3867, 0.9088, -0.8634, -0.6677, -0.7275, -0.462, 

In [11]:
print(Y)
len(Y)

[0.4624, -0.24, -0.2242, -0.5741, 0.3295, -0.2155, -0.1534, 0.2938, -0.1801, 0.3139, -0.2072, -0.588, 0.2145, -0.1869, -0.1405, -0.1127, 0.7635, -0.2176, -0.5483, -0.2165, -0.5388, -0.8916, -0.7081, -0.4481, -0.1563, -0.43, -0.2186, -0.5762, -0.0101, -0.1072, -0.1866, -0.2205, 0.3212, -0.2337, -0.1505, 0.2991, -0.1787, 0.0658, -0.0818, -0.1143, -0.0752, -0.2282, -0.1008, -0.0484, 0.0945, -0.1598, -0.3383, -0.2395, -0.2825, 0.735, -0.8993, -0.3717, -0.1295, -0.4939, -0.1902, -0.2091, -0.4835, -0.8107, -0.0007, -0.1781, -0.449, -0.1509, -0.9266, 0.1974, 0.2264, -0.1806, -0.2277, -0.4922, -0.2558, -0.0754, -0.2574, -0.2051, -0.1979, -0.4657, -0.1473, -0.147, -0.1161, 1.0, -0.4297, -0.5174, 0.0004, -0.2419, -0.2804, 0.0287, -0.6571, -1.0, -0.0704, 0.344, -0.0454, 0.9318, -0.0113, -0.2424, 0.1311, -0.4673, -0.2163, -0.3008, -0.2158, -0.2219, -0.3553, -0.2833]


100

In [ ]:
nn.run((np.asfarray(all_values[1:])/255.0*.99)+0.01)